In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/train.csv', index_col='PassengerId')        #make id-column the index column
test = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/test.csv', index_col='PassengerId')          #id is not a column anymore, instead acts as an index
submission = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/sample_submission.csv', index_col='PassengerId')

In [ ]:
train.head()
#train.shape
#test.shape
#submission.head()

In [ ]:
train.Survived.sum()
train.describe()

In [ ]:
print(train.isna().sum())
print("age missing %:", train.isna().sum()[4]/train.shape[0]*100)

### Imputation of Missing Values

In [ ]:
print(test.isna().sum())

In [ ]:
train=train.drop(columns=["Name", "Cabin", "Ticket"])
test=test.drop(columns=["Name", "Cabin", "Ticket"])
train = train.dropna()
#test= test.dropna()
train.shape

In [ ]:
#interpolate train
#train=train.interpolate()
#train.isna().sum()
#train['Age']=train['Age'].fillna(train['Age'].mean())    
#train["Embarked"]=train["Embarked"].replace(np.NaN,"S")
train.isna().sum()

In [ ]:
train.isna().sum()

In [ ]:
#impute train
#train['Age']=train['Age'].fillna(train['Age'].mean())    
#train['Fare']=train['Fare'].fillna(train['Fare'].mean())    
#train.isna().sum()
#train["Embarked"].value_counts(normalize="True")
#train.isna().sum()
#train["Embarked"].value_counts(normalize="True")
#train["Embarked"]
#train["Embarked"]=train["Embarked"].replace(np.NaN,"S")
#train.isna().sum()


#### take log of fare and medians instead of means

In [ ]:
#impute test
test['Age']=test['Age'].fillna(test['Age'].mean())    
test['Fare']=test['Fare'].fillna(test['Fare'].mean())    
test.isna().sum()
test["Embarked"].value_counts(normalize="True")
test.isna().sum()
test["Embarked"].value_counts(normalize="True")
test["Embarked"]
test["Embarked"]=test["Embarked"].replace(np.NaN,"S")
test.isna().sum()
#test["Embarked"].value_counts(normalize="True")
#len(test)

In [ ]:
print("Females survived:", len(train.loc[train.Sex == "female"][train.Survived == 1])/len(train))
print("Males survived:",len(train.loc[train.Sex == "male"][train.Survived == 1])/len(train))
print("Survival Rate:", len(train.loc[train.Survived ==1])/len(train))

In [ ]:
print("1st Class Survived", len(train.loc[train.Pclass == 1][train.Survived == 1])/len(train.loc[train.Pclass == 1]))
print("2nd Class Survived", len(train.loc[train.Pclass == 2][train.Survived == 1])/len(train.loc[train.Pclass == 2]))
print("3rd Class Survived", len(train.loc[train.Pclass == 3][train.Survived == 1])/len(train.loc[train.Pclass == 3]))

In [ ]:
print("Age under 5 survived", len(train.loc[train.Age <= 5][train.Survived == 1])/len(train.loc[train.Age <= 5]))

In [ ]:
len(train.loc[train.Age <= 5])/len(train)

In [ ]:
train.dtypes

In [ ]:
train.isna().any()
test.isna().any()

In [ ]:
from sklearn import preprocessing
enc = preprocessing.LabelEncoder()

for c in train.columns:
    if train[c].dtype=='object': 
        enc = preprocessing.LabelEncoder()
        enc.fit(list(train[c].values) + list(test[c].values))   
        #print(list(le.classes_))
        train[c] = enc.transform(train[c].values)
        test[c] = enc.transform(test[c].values)

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

pd.set_option('display.max_columns', 200)

def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [ ]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 5, 7, 10],
        'learning_rate': [0.01, 0.02, 0.05]    
        }

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train.drop(columns=["Survived"]), train['Survived'], random_state=1)

In [ ]:
!nvidia-smi

In [ ]:
y_train

In [ ]:
%%time 
folds = 3
param_comb = 6

X = X_train
Y = y_train

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

xgb = XGBClassifier(learning_rate=0.02, n_estimators=1000, objective='binary:logistic',
                    silent=True, nthread=6, tree_method='gpu_hist', eval_metric='auc')

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X,Y), verbose=3, random_state=1001 )

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X, Y)
timer(start_time) # timing ends here for "start_time" variable

In [ ]:
preds = random_search.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(preds, y_test)

In [ ]:
%%time

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

grid_search = {'criterion': ['entropy', 'gini'],
               'max_depth': [2],
               'max_features': ['auto', 'sqrt'],
               'min_samples_leaf': [4, 6, 8],
               'min_samples_split': [5, 7,10],
               'n_estimators': [20]}

clf = RandomForestClassifier()
model = GridSearchCV(estimator = clf, param_grid = grid_search, 
                               cv = 4, verbose= 5, n_jobs = -1)
model.fit(X_train,y_train)

predictionforest = model.best_estimator_.predict(X_test)
print(confusion_matrix(y_test,predictionforest))
print(classification_report(y_test,predictionforest))
acc4 = accuracy_score(y_test,predictionforest)
acc4

In [ ]:
#type(estimators)

In [ ]:
estimators=[("rf_tuned", model), ("xgboost_tuned", random_search)]

#create our voting classifier, inputting all models
from sklearn.ensemble import VotingClassifier
ensemble = VotingClassifier(estimators, voting="hard")

ensemble.fit(X_train, y_train)
preds = ensemble.predict(y_test)

from sklearn.metrics import accuracy_score
#accuracy_score(preds, y_test)

In [ ]:
ensemble.fit(X_train, y_train)
preds = ensemble.predict(y_test)

from sklearn.metrics import accuracy_score
#accuracy_score(preds, y_test)

In [ ]:
%%time 
folds = 3
param_comb = 6

X = train.drop(columns=["Survived"])
Y = train['Survived']

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

xgb = XGBClassifier(learning_rate=0.02, n_estimators=1000, objective='binary:logistic',
                    silent=True, nthread=6, tree_method='gpu_hist', eval_metric='auc')

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X,Y), verbose=3, random_state=1001 )

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X, Y)
timer(start_time) # timing ends here for "start_time" variable

In [ ]:
submission['Survived'] = random_search.predict(test)
submission.to_csv('xgboost_random_search_tune.csv')

In [ ]:
submission.value_counts()